<a href="https://colab.research.google.com/github/ChaithanyaSaiB/UMBC-DATA606-Capstone/blob/main/notebooks/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importing necessary libraries
import pandas as pd
import re
import os
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
# Import the tqdm library for progress bar
from tqdm import tqdm

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load CNN health data
cnnhealth = pd.read_csv("/content/cnnhealth.txt", sep=r"(?<!\s)[|](?!\s)", names=['ID', 'DateTime', 'Article Title and Link'], engine="python")
cnnhealth.head()

,ID,DateTime,Article Title and Link
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [ ]:
# Extract all URLs from article titles and links
cnnhealth['URL'] = [re.findall(r'https?://\S+', text) for text in cnnhealth['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = cnnhealth['Article Title and Link'][cnnhealth['URL'].apply(len) != 1].index
cnnhealth.drop(drop_indices, inplace=True)

# Extract CNN URL for articles
cnnhealth['URL'] = [url[0] for url in cnnhealth['URL']]

In [ ]:
# Function to fetch content from a URL
def fetch_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print("Error fetching content:", e)
        return None

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content):
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        article_body = soup.findAll('p', class_='paragraph')
        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
cnnhealth['Content'] = extract_content_with_progress(cnnhealth['URL'])

Extracting content:   0%|          | 15/3299 [00:14<40:11,  1.36it/s]/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Extracting content:   3%|▎         | 113/3299 [01:15<27:07,  1.96it/s]

Error fetching content: Failed to parse: https://t…


Extracting content:   6%|▌         | 200/3299 [02:04<41:49,  1.23it/s]

Error fetching content: Failed to parse: http://t.c…


Extracting content:   7%|▋         | 246/3299 [02:29<24:43,  2.06it/s]

...

Error fetching content: HTTPConnectionPool(host='t.', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f139e40>: Failed to resolve 't' ([Errno -2] Name or service not known)"))


Extracting content:  81%|████████  | 2662/3299 [26:34<00:13, 47.87it/s]

Error fetching content: HTTPConnectionPool(host='t.c', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7b450f13b610>: Failed to resolve 't.c' ([Errno -2] Name or service not known)"))


Extracting content: 100%|██████████| 3299/3299 [26:55<00:00,  2.04it/s]

Error fetching content: HTTPConnectionPool(host='on.cnn.com', port=80): Max retries exceeded with url: /SO1PxT (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7b450dbd85b0>, 'Connection to on.cnn.com timed out. (connect timeout=10)'))


In [ ]:
# Drop rows with missing content
cnnhealth.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
cnnhealth['Content'] = [" ".join([p_tag.text.strip() for p_tag in content]) for content in cnnhealth['Content']]

In [ ]:
# Clean the content data
def clean_text(text):
    text = re.sub(r'-', ' ', text)  # Substitute hyphens with empty spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert text to lowercase
    return text

cnnhealth['Content'] = cnnhealth['Content'].apply(clean_text)

In [ ]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [ ]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

cnnhealth['Content'] = cnnhealth['Content'].apply(preprocess_text)

In [ ]:
# Save preprocessed data to CSV
cnnhealth.to_csv('cnnhealth_preprocessed.csv', index=False)

In [4]:
# Define the directory path containing the .txt files
directory = '/content'

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Read the .txt file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Extract filename without extension
        try:
            df = pd.read_csv(os.path.join(directory, filename),
                             sep=r"(?<!\s)[|](?!\s)",
                             names=['ID', 'DateTime', 'Article Title and Link'],
                             engine="python")
            # Add the DataFrame to the list
            dfs.append(df)
            print(f"DataFrame '{df_name}' loaded with {len(df)} rows.")
        except UnicodeDecodeError:
            print(f"Error reading {filename}: UnicodeDecodeError. Skipping this file.")

# Concatenate all DataFrames in the list into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:")
print(combined_df)

DataFrame 'nprhealth' loaded with 4837 rows.
DataFrame 'cnnhealth' loaded with 4061 rows.
DataFrame 'cbchealth' loaded with 3741 rows.
DataFrame 'everydayhealth' loaded with 3239 rows.
DataFrame 'latimeshealth' loaded with 4171 rows.
Combined DataFrame:
                       ID                        DateTime  \
0      547824165185536000  Wed Dec 24 18:40:33 +0000 2014   
1      547763897638199296  Wed Dec 24 14:41:04 +0000 2014   
2      547671560471859200  Wed Dec 24 08:34:09 +0000 2014   
3      547671553609981952  Wed Dec 24 08:34:08 +0000 2014   
4      547671544680312834  Wed Dec 24 08:34:06 +0000 2014   
...                   ...                             ...   
20044  144952345170690052  Fri Dec 09 01:31:46 +0000 2011   
20045  144936916838649857  Fri Dec 09 00:30:28 +0000 2011   
20046  144917037414756354  Thu Dec 08 23:11:28 +0000 2011   
20047  144881597869592576  Thu Dec 08 20:50:39 +0000 2011   
20048  144861369597050880  Thu Dec 08 19:30:16 +0000 2011   

             

In [5]:
# Extract all URLs from article titles and links in combined_df
combined_df['URL'] = [re.findall(r'https?://\S+', text) for text in combined_df['Article Title and Link']]

# Drop rows with multiple or no URLs
drop_indices = combined_df['Article Title and Link'][combined_df['URL'].apply(len) != 1].index
combined_df.drop(drop_indices, inplace=True)

# Extract URL for articles
combined_df['URL'] = [url[0] for url in combined_df['URL']]


In [6]:
!pip install fake-useragent

In [7]:
# Calculate the total number of rows
total_rows = len(combined_df)

# Calculate the size of each split
split_size = total_rows // 10

# Define the start and end indices for each split
split_1 = combined_df.iloc[:split_size].copy()
split_2 = combined_df.iloc[split_size:2*split_size].copy()
split_3 = combined_df.iloc[2*split_size:3*split_size].copy()
split_4 = combined_df.iloc[3*split_size:4*split_size].copy()
split_5 = combined_df.iloc[4*split_size:5*split_size].copy()
split_6 = combined_df.iloc[5*split_size:6*split_size].copy()
split_7 = combined_df.iloc[6*split_size:7*split_size].copy()
split_8 = combined_df.iloc[7*split_size:8*split_size].copy()
split_9 = combined_df.iloc[8*split_size:9*split_size].copy()
split_10 = combined_df.iloc[9*split_size:].copy()

In [8]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# Create a UserAgent instance
ua = UserAgent()

# Function to fetch content from a URL
def fetch_content(url):
    try:
        # Define user-agent header using a random user agent
        headers = {'User-Agent': ua.random}

        # Create a session to handle cookies and maintain connection state
        with requests.Session() as session:
            response = session.get(url, headers=headers, timeout=4, allow_redirects=True)
            response.raise_for_status()  # Raise an error for 4xx and 5xx status codes
            final_url = response.url  # Get the final URL after following redirects
            website_name = get_website_name(final_url)
            return (response.text, website_name)
    except Exception as e:
        print("Error fetching content:", e)
        return (None, None)

# Function to extract website name from URL
def get_website_name(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content_with_website_name):
    html_content, website_name = html_content_with_website_name
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        if soup:
            if website_name == 'www.cnn.com':
                article_body = soup.findAll('p', class_='paragraph')
            elif website_name == 'www.health.usnews.com':
                article_body = soup.findAll('p')
            elif website_name == 'www.latimes.com':
                article_body = soup.findAll('p')
            elif website_name == 'www.npr.org':
                storytext_div = soup.find('div', id='storytext')
                if storytext_div:
                    article_body = storytext_div.find_all('p')
                else:
                    article_body = None
            elif website_name == 'www.cbc.ca':
                storytext_div = soup.find('div', class_='story')
                if storytext_div:
                    article_body = storytext_div.find_all('p')
                else:
                    article_body = None
            else:
                article_body = None

            if article_body:
                return article_body
            else:
                return None
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
def extract_content_with_progress(url_list):
    content_list = []
    for url in tqdm(url_list, desc='Extracting content'):
        content = extract_body(fetch_content(url))
        content_list.append(content)
    return content_list

# Apply web scraping to extract content and store it in a new column
#combined_df['Content'] = extract_content_with_progress(combined_df['URL'])

In [8]:
split_1['Content'] = extract_content_with_progress(split_1['URL'])
split_1.to_csv('split_1_preprocessed.csv', index=False)

Extracting content:   8%|▊         | 138/1797 [02:21<31:40,  1.15s/it]

Error fetching content: 404 Client Error: Not Found for url: https://feeds.npr.org/rss/index.html?utm_source=twitter.com&utm_campaign=health&utm_medium=social&utm_term=nprnews


Extracting content:  11%|█         | 198/1797 [03:20<20:59,  1.27it/s]

Error fetching content: 404 Client Error: Not Found for url: https://interns-npr.icims.com/jobs/2239/winter-spring-2015:-science-desk,-editorial-or-multimedia/job


Extracting content:  16%|█▌        | 280/1797 [04:43<20:35,  1.23it/s]

...

Extracting content:  92%|█████████▏| 1646/1797 [29:03<02:20,  1.08it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/2014/05/07/310287526/settlement-fund-would-compensate-2012-meningitis-victims?utm_source=twitter.com&utm_campaign=health&utm_medium=social&utm_term=nprnews?utm_source=twitter.com&utm_campaign=health&utm_medium=social&utm_term=nprnews


Extracting content:  96%|█████████▌| 1717/1797 [30:11<01:21,  1.02s/it]

Error fetching content: 404 Client Error: Not Found for url: https://feeds.npr.org/rss/index.html?utm_source=twitter.com&utm_campaign=health&utm_medium=social&utm_term=nprnews


Extracting content: 100%|██████████| 1797/1797 [31:32<00:00,  1.05s/it]


In [9]:
split_2['Content'] = extract_content_with_progress(split_2['URL'])
split_2.to_csv('split_2_preprocessed.csv', index=False)

Extracting content:   5%|▍         | 89/1797 [01:32<26:32,  1.07it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/2014/03/25/294562423/some-will-get-extra-time-to-enroll-in-health-insurance?ft=1&f=1128&utm_source=health&utm_campaign=nprnews&utm_medium=twitter?ft=1&f=1128&utm_source=health&utm_campaign=nprnews&utm_medium=twitter


Extracting content:  11%|█         | 198/1797 [03:22<24:49,  1.07it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/2014/03/07/287115235/sports-medicine-pioneer-fank-jobe-dies-at-88?ft=1&f=1128&utm_source=health&utm_campaign=nprnews&utm_medium=twitter?ft=1&f=1128&utm_source=health&utm_campaign=nprnews&utm_medium=twitter


Extracting content:  12%|█▏        | 209/1797 [03:33<24:12,  1.09it/s]

...

Extracting content:  94%|█████████▍| 1693/1797 [25:56<01:09,  1.50it/s]

Error fetching content: 410 Client Error: Gone for url: http://bitly.com/bundles/scotthensley/l


Extracting content:  98%|█████████▊| 1755/1797 [26:47<00:30,  1.37it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/sandbox/mobilebeta/


Extracting content: 100%|██████████| 1797/1797 [27:26<00:00,  1.09it/s]


In [10]:
split_3['Content'] = extract_content_with_progress(split_3['URL'])
split_3.to_csv('split_3_preprocessed.csv', index=False)

Extracting content:   6%|▌         | 111/1797 [01:37<22:08,  1.27it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/2013/05/14/183842832/angelina-jolie-says-she-had-double-mastectomy?ft=1&f=1128&sc=tw?ft=1&f=1128&sc=tw


Extracting content:  10%|▉         | 173/1797 [02:33<20:41,  1.31it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.npr.org/2013/04/30/180131300/obama-administration-challenges-morning-after-pill-ruling?ft=1&f=1128&sc=tw?ft=1&f=1128&sc=tw


Extracting content:  15%|█▌        | 270/1797 [03:59<16:35,  1.53it/s]

...

Extracting content: 100%|█████████▉| 1793/1797 [21:46<00:05,  1.48s/it]

Error fetching content: HTTPConnectionPool(host='upwave.co', port=80): Max retries exceeded with url: /1kTcVP2 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7b6089d97490>, 'Connection to upwave.co timed out. (connect timeout=4)'))


Extracting content: 100%|█████████▉| 1794/1797 [21:46<00:03,  1.18s/it]

Error fetching content: 410 Client Error: Gone for url: http://globalpublicsquare.blogs.cnn.com/2014/04/19/how-u-s-is-lagging-on-quality-of-life/


Extracting content: 100%|██████████| 1797/1797 [21:47<00:00,  1.37it/s]


In [7]:
split_4['Content'] = extract_content_with_progress(split_4['URL'])
split_4.to_csv('split_4_preprocessed.csv', index=False)

Extracting content:   0%|          | 3/1797 [00:01<14:07,  2.12it/s]

Error fetching content: 410 Client Error: Gone for url: http://thechart.blogs.cnn.com/2014/04/18/5-studies-you-may-have-missed-27/


Extracting content:   1%|          | 10/1797 [00:06<19:33,  1.52it/s]

Error fetching content: 410 Client Error: Gone for url: http://thechart.blogs.cnn.com/2014/04/17/salmonella-cases-down-but-watch-out-for-other-foodborne-bacteria/


Extracting content:   1%|          | 16/1797 [00:13<33:14,  1.12s/it]

...

Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/nVFU2R2
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/27WwpJC
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/7NTAYfF
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/PlSolEC
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/lV4druw
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/PglEhGm
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/Z44lnda
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/XXSvgCI
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/u6Tvvkq
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/49tUdJ4
Error fetching content: 500 Server Error: Domain Not Found f

Extracting content:  99%|█████████▉| 1781/1797 [11:38<00:00, 24.07it/s]

Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/v26T7MR
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/Sbl1GtT
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/AzSVJys
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/E9CVo7E
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/Ojai5rg
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/x1qXrWG
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/01Tdt1o


Extracting content: 100%|██████████| 1797/1797 [11:39<00:00,  2.57it/s]


Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/S3jSB9G
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/K0qXC0K
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/iu4CPrm
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/suMAN5K
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/cvrq7En
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/wpWyjvA
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/0nbdAnP
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/qmN1lF9
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/sv9WVaa
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/ggQb92z
Error fetching content: 500 Server Error: Domain Not Found f

In [8]:
split_5['Content'] = extract_content_with_progress(split_5['URL'])
split_5.to_csv('split_5_preprocessed.csv', index=False)

Extracting content:   0%|          | 0/1797 [00:00<?, ?it/s]

Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/p5QWwSw


Extracting content:   0%|          | 2/1797 [00:08<2:00:13,  4.02s/it]

Error fetching content: HTTPConnectionPool(host='www.turner.com', port=80): Max retries exceeded with url: /careers/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c2b31e7af80>, 'Connection to www.turner.com timed out. (connect timeout=4)'))
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/TYXUoPJ
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/JIdXuik
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/nKeURKu
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/kogpDMw
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/Wm6UJb0
Error fetching content: 500 Server Error: Domain Not Found for url: http://at.cnn.com/sp6Iu8M


Extracting content:   1%|          | 21/1797 [00:09<07:55,  3.74it/s]

...

Error fetching content: HTTPConnectionPool(host='cnw.ca', port=80): Max retries exceeded with url: /AywWG (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c2b163bbcd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


Extracting content: 100%|█████████▉| 1790/1797 [17:41<00:05,  1.24it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/R7L7O%E2%80%A6


Extracting content: 100%|██████████| 1797/1797 [17:50<00:00,  1.68it/s]


Error: need to escape, but no escapechar set

In [9]:
split_5.to_csv('split_5_preprocessed.csv', index=False, escapechar='\\')

In [9]:
split_6['Content'] = extract_content_with_progress(split_6['URL'])
split_6.to_csv('split_6_preprocessed.csv', index=False, escapechar='\\')

Extracting content:   0%|          | 4/1797 [00:04<31:07,  1.04s/it]

Error fetching content: 404 Client Error: Not Found for url: http://pbs.twimg.com/media/Bz_72tkIQAEDQ4W.jpg


Extracting content:   4%|▎         | 63/1797 [01:14<39:23,  1.36s/it]

Error fetching content: HTTPConnectionPool(host='buzz.mw', port=80): Max retries exceeded with url: /bc432_f (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7edaf20a0910>: Failed to resolve 'buzz.mw' ([Errno -2] Name or service not known)"))


Extracting content:   5%|▌         | 97/1797 [01:50<27:08,  1.04it/s]

...

Extracting content:  88%|████████▊ | 1582/1797 [30:21<07:08,  1.99s/it]

Error fetching content: HTTPSConnectionPool(host='www.cbc.ca', port=443): Read timed out. (read timeout=4)


Extracting content:  96%|█████████▌| 1725/1797 [32:59<01:10,  1.02it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/news/health/bike-helmet-mandate-for-all-sought-by-pediatricians-1.2288888?cmp=rss


Extracting content: 100%|██████████| 1797/1797 [34:20<00:00,  1.15s/it]


In [11]:
split_7['Content'] = extract_content_with_progress(split_7['URL'])
split_7.to_csv('split_7_preprocessed.csv', index=False, escapechar='\\')

Extracting content:   8%|▊         | 137/1797 [02:39<28:33,  1.03s/it]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/news/health/story/2013/09/13/testosterone-estrogen-men.html?cmp=rss


Extracting content:   8%|▊         | 152/1797 [02:57<29:04,  1.06s/it]

Error fetching content: 404 Client Error: Not Found for url: https://www.cbc.ca/player/News/Health/ID/2405542163?cmp=rss


Extracting content:   9%|▊         | 156/1797 [03:01<25:13,  1.08it/s]

...

Extracting content:  98%|█████████▊| 1765/1797 [33:02<02:17,  4.29s/it]

Error fetching content: HTTPConnectionPool(host='preview.everydayhealth.com', port=80): Max retries exceeded with url: /healthtalk/expert-advice-on-living-well-with-diabetes.aspx (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ed9bdacd9c0>, 'Connection to preview.everydayhealth.com timed out. (connect timeout=4)'))


Extracting content:  98%|█████████▊| 1766/1797 [33:03<01:35,  3.08s/it]

Error fetching content: HTTPConnectionPool(host='qa1.everydayhealth.com', port=80): Max retries exceeded with url: /emotional-health/0614/what-do-your-shoes-say-about-your-personality.aspx?xid=tw_everydayhealth_sf (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7ed9bdacd150>: Failed to resolve 'qa1.everydayhealth.com' ([Errno -2] Name or service not known)"))


Extracting content: 100%|██████████| 1797/1797 [33:25<00:00,  1.12s/it]


In [13]:
split_7.to_csv('split_7_preprocessed.csv', index=False, escapechar='\\')

In [12]:
split_8['Content'] = extract_content_with_progress(split_8['URL'])
split_8.to_csv('split_8_preprocessed.csv', index=False)

Extracting content:   0%|          | 3/1797 [00:00<08:20,  3.58it/s]

Error fetching content: 404 Client Error: Not Found for url: https://www.everydayhealth.com/blog/health-in-the-news/2011/11/30/are-you-living-in-one-of-america%E2%80%99s-saddest-cities/?xid=tw_everydayhealth_20111130_sadcity


Extracting content:   1%|          | 13/1797 [00:05<13:28,  2.21it/s]

Error fetching content: 410 Client Error: Gone for url: https://www.everydayhealth.com/diet-and-nutrition/0308/fruits-veggies-can-be-beauty-tools.aspx?xid=tw_everydayhealth_sf


Extracting content:   1%|▏         | 26/1797 [00:26<2:01:21,  4.11s/it]

...

Extracting content:  90%|█████████ | 1622/1797 [11:14<01:01,  2.83it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/kNe8Pa%E2%80%A6


Extracting content: 100%|█████████▉| 1795/1797 [13:04<00:01,  1.65it/s]

Error fetching content: Failed to parse: http://t…


Extracting content: 100%|██████████| 1797/1797 [13:04<00:00,  2.29it/s]


In [13]:
split_9['Content'] = extract_content_with_progress(split_9['URL'])
split_9.to_csv('split_9_preprocessed.csv', index=False)

Extracting content:   0%|          | 6/1797 [00:03<11:28,  2.60it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/x%E2%80%A6


Extracting content:   1%|          | 10/1797 [00:05<12:45,  2.33it/s]

Error fetching content: 404 Client Error: Not Found for url: https://t.co/T5g%E2%80%A6


Extracting content:   1%|          | 18/1797 [00:08<11:44,  2.53it/s]

...

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFP2R (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c2a0e6be440>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content: 100%|█████████▉| 1796/1797 [49:59<00:02,  2.78s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFU8z (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c2a0e07dd20>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content: 100%|██████████| 1797/1797 [50:03<00:00,  1.67s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFQf1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7c2a0e07dab0>, 'Connection to lati.ms timed out. (connect timeout=4)'))


In [10]:
split_10['Content'] = extract_content_with_progress(split_10['URL'])
split_10.to_csv('split_10_preprocessed.csv', index=False)

Extracting content:   0%|          | 1/1799 [00:04<2:00:39,  4.03s/it]

Error fetching content: HTTPConnectionPool(host='lati.ms', port=80): Max retries exceeded with url: /lFOEJ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7edab518eef0>, 'Connection to lati.ms timed out. (connect timeout=4)'))


Extracting content:   0%|          | 2/1799 [00:04<54:13,  1.81s/it]  

Error fetching content: 404 Client Error: Not Found for url: https://t.co/%E2%80%A6


Extracting content:   0%|          | 3/1799 [00:08<1:24:17,  2.82s/it]

...

Extracting content:  99%|█████████▉| 1788/1799 [26:45<00:04,  2.52it/s]

Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/V871HpDYMLM/la-he-breast-cancer-drugs-20111208,0,5300399.story


Extracting content: 100%|█████████▉| 1791/1799 [26:47<00:03,  2.62it/s]

Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/AYBnh01qkuo/os-carb-diet-cancer-20111209,0,7459472.story
Error fetching content: 404 Client Error: Not Found for url: http://feeds.latimes.com/~r/latimes/features/health/~3/maIYoUqs0po/sns-rt-us-states-rankingtre7b52dp-20111206,0,2739896.story


Extracting content: 100%|██████████| 1799/1799 [26:53<00:00,  1.11it/s]


In [12]:
split_10.to_csv('split_10_preprocessed.csv', index=False)

In [14]:
# List to store DataFrames of all splits
dfs = []

# Read each CSV file and append its DataFrame to the list
for i in range(1, 11):
    filename = f'/content/split_{i}_preprocessed.csv'
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

In [15]:
combined_df['Content'] = combined_df['Content'].apply(lambda x: [x])

In [16]:
# Drop rows with missing content
combined_df.dropna(subset=['Content'], inplace=True)

# Combine content paragraphs into a single string
combined_df['Content'] = [" ".join([str(p_tag).strip() for p_tag in content]) for content in combined_df['Content']]

In [18]:
combined_df.head()

,ID,DateTime,Article Title and Link,URL,Content
0,547824165185536000,Wed Dec 24 18:40:33 +0000 2014,Would You Like Health Insurance With Those Sto...,http://n.pr/1CKulog,[<p>\n Need a gift for a 20-som...
1,547763897638199296,Wed Dec 24 14:41:04 +0000 2014,New Blood Donation Rules Would Still Exclude M...,http://n.pr/1CJ9IsG,"[<p>On Tuesday, the Food and Drug Administrati..."
2,547671560471859200,Wed Dec 24 08:34:09 +0000 2014,Obama Administration Downplays Court Challenge...,http://n.pr/1CHaDd6,[<p>\n Elisa Carrero assists Ju...
3,547671553609981952,Wed Dec 24 08:34:08 +0000 2014,"Christmas In Liberia: Ebola Fears, No Snow, Ho...",http://n.pr/1CHaAxQ,[<p>\n A man peddles plastic Ch...
4,547671544680312834,Wed Dec 24 08:34:06 +0000 2014,Costly Hepatitis C Drugs Threaten To Bust Pris...,http://n.pr/1CHaCFX,[<p>\n Dr. Michael Poshkus trea...


In [19]:
# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Substitute hyphens with empty spaces
    text = re.sub(r'-', ' ', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

# Apply the clean_text function to the 'Content' column
combined_df['Content'] = combined_df['Content'].apply(clean_text)

In [20]:
# Convert to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None  # Use default POS for lemmatization

In [21]:
# Tokenize and preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize text
    lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
    pos_tags = nltk.pos_tag(tokens)  # Get part-of-speech tags
    for i, (token, tag) in enumerate(pos_tags):
        pos = get_wordnet_pos(tag)  # Convert NLTK POS tags to WordNet POS tags
        if pos:
            tokens[i] = lemmatizer.lemmatize(token, pos=pos)  # Lemmatize tokens
        else:
            tokens[i] = lemmatizer.lemmatize(token)  # Use default POS for lemmatization
    stop_words = set(stopwords.words('english'))  # Get stopwords
    custom_stopwords = [    # Custom stopwords
    "patient", "doctor", "say", "year", "state", "day", "need", "come", "well",
    "make", "think", "know", "go", "use", "one", "like", "people", "may",
    "many", "still", "even", "two", "way", "good", "much", "back", "new",
    "time", "first", "really",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
    ]
    tokens = [token for token in tokens if token not in stop_words and token not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

combined_df['Content'] = combined_df['Content'].apply(preprocess_text)

In [22]:
combined_df

,ID,DateTime,Article Title and Link,URL,Content
0,547824165185536000,Wed Dec 24 18:40:33 +0000 2014,Would You Like Health Insurance With Those Sto...,http://n.pr/1CKulog,gift something kid age family health plan juan...
1,547763897638199296,Wed Dec 24 14:41:04 +0000 2014,New Blood Donation Rules Would Still Exclude M...,http://n.pr/1CJ9IsG,tuesday food drug administration recommend cha...
2,547671560471859200,Wed Dec 24 08:34:09 +0000 2014,Obama Administration Downplays Court Challenge...,http://n.pr/1CHaDd6,elisa carrero assist julian gauiria paterson n...
3,547671553609981952,Wed Dec 24 08:34:08 +0000 2014,"Christmas In Liberia: Ebola Fears, No Snow, Ho...",http://n.pr/1CHaAxQ,man peddle plastic christmas tree light downto...
4,547671544680312834,Wed Dec 24 08:34:06 +0000 2014,Costly Hepatitis C Drugs Threaten To Bust Pris...,http://n.pr/1CHaCFX,dr michael poshkus treat inmates hepatitis joh...
...,...,...,...,...,...
17967,144952345170690052,Fri Dec 09 01:31:46 +0000 2011,"Shift work: Good for your pocketbook, bad for ...",http://lat.ms/srI28g,job remain gainfully employ take toll health e...
17968,144936916838649857,Fri Dec 09 00:30:28 +0000 2011,Hold the pasta: Starchy food linked to breast ...,http://lat.ms/vopkPz,post correct see note bottom detail another re...
17969,144917037414756354,Thu Dec 08 23:11:28 +0000 2011,"Chronic pain is common child complaint, study ...",http://lat.ms/sqbbYO,chronic pain usually problem affect adult stud...
17970,144881597869592576,Thu Dec 08 20:50:39 +0000 2011,"Sporadic low-carb diets may help lower weight,...",http://lat.ms/vz4beE,intermittent low carb diet could standard low ...


In [42]:
combined_df[combined_df['Content'].apply(len) <= 50].index

Index([   58,   137,   197,   210,   279,   351,   467,   597,   624,   865,
       ...
       17928, 17930, 17934, 17935, 17936, 17958, 17959, 17960, 17962, 17963],
      dtype='int64', length=6156)

In [41]:
combined_df['Content'].iloc[210]

'nan'

In [23]:
# Save preprocessed data to CSV
combined_df.to_csv('combined_preprocessed.csv', index=False)